# Feature Aggregator - Полный Pipeline для Global ML Model

**Цель**: Объединить все признаки и создать финальные датасеты для ML и Backtesting.

## Критические правила нормализации:

### ✅ ЧТО ВКЛЮЧАЕМ В ML ДАТАСЕТ:
- `log_return` - логарифмическая доходность
- `volume_zscore_*` - Z-score объема
- `dist_to_sma_*`, `dist_to_ema_*` - расстояние до MA в процентах
- `rsi_14` - RSI (0-100)
- `momentum_*` - нормализованный momentum
- `overnight_gap` - гэп открытия
- `day_of_week`, `day_of_month` - календарные признаки
- `ticker_id`, `sector_id` - метаданные для глобальной модели

### ❌ ЧТО НЕ ВКЛЮЧАЕМ В ML ДАТАСЕТ:
- `open`, `high`, `low`, `close` - абсолютные цены
- `volume` - абсолютный объем  
- `sma_*`, `ema_*` - абсолютные значения MA
- `vp_poc`, `vp_va_high`, `vp_va_low` - абсолютные уровни Volume Profile

## Выходные файлы:
- `data/processed_ml/{TICKER}_ml_features.parquet` - для ML
- `data/backtest/{TICKER}_price_data.parquet` - для бэктеста


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к корневой директории ML
sys.path.insert(0, str(Path.cwd().parent))

# Импорт модулей признаков
from features import (
    build_all_features,
    process_all_tickers,
    get_ml_feature_columns,
    FORBIDDEN_ML_COLUMNS,
    VOLUME_FEATURE_COLUMNS,
    TREND_FEATURE_COLUMNS,
    CALENDAR_FEATURE_COLUMNS
)
from config import get_tickers_by_liquidity, get_ticker_metadata

print("✅ Библиотеки и модули загружены")


## Конфигурация путей
-Скорее всего будет ошибка, поскольку данные берутся по другому пути

In [ ]:
# === КОНФИГУРАЦИЯ ПУТЕЙ ===
ML_ROOT = Path.cwd().parent

# Входные данные (OHLCV с log_returns из этапа 01)
DATA_DIR = ML_ROOT / "data" / "processed"

# Выходные данные - ДВА отдельных каталога!
OUTPUT_ML_DIR = ML_ROOT / "data" / "processed_ml"      # Для ML модели (нормализованные)
OUTPUT_BACKTEST_DIR = ML_ROOT / "data" / "backtest"    # Для бэктеста (сырые цены)

# Создаем директории
OUTPUT_ML_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_BACKTEST_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Источник данных: {DATA_DIR}")
print(f"📂 ML выход: {OUTPUT_ML_DIR}")
print(f"📂 Backtest выход: {OUTPUT_BACKTEST_DIR}")

# Проверяем доступные тикеры
available_files = list(DATA_DIR.glob("*_ohlcv_returns.parquet"))
available_tickers = [f.stem.replace('_ohlcv_returns', '') for f in available_files]
print(f"\n📋 Доступные тикеры ({len(available_tickers)}): {available_tickers}")


## Демонстрация на одном тикере


In [ ]:
# Демонстрация на одном тикере
ticker = 'SBER'

# Загрузка исходных данных
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
print(f"📥 Загружено: {len(df)} записей для {ticker}")
print(f"\nИсходные столбцы ({len(df.columns)}): {df.columns.tolist()}")

# Расчет ВСЕХ признаков
ml_features, backtest_data = build_all_features(df, ticker)

print(f"\n{'='*60}")
print(f"📊 РЕЗУЛЬТАТЫ ДЛЯ {ticker}")
print(f"{'='*60}")


In [ ]:
# === ML FEATURES (нормализованные) ===
print(f"✅ ML Features: {len(ml_features.columns)} столбцов")
print(f"\nСтолбцы ML: {ml_features.columns.tolist()}")
print(f"\nПример данных ML:")
display(ml_features.tail(10))

# Проверка: НЕ должно быть запрещенных столбцов!
forbidden_found = [col for col in ml_features.columns if col in FORBIDDEN_ML_COLUMNS]
if forbidden_found:
    print(f"\n❌ ОШИБКА: Найдены запрещенные столбцы: {forbidden_found}")
else:
    print(f"\n✅ Проверка пройдена: ML features не содержат абсолютных значений")


In [ ]:
# === BACKTEST DATA (сырые цены) ===
print(f"✅ Backtest Data: {len(backtest_data.columns)} столбцов")
print(f"\nСтолбцы Backtest: {backtest_data.columns.tolist()}")
print(f"\nПример данных Backtest:")
display(backtest_data.tail(10))


In [ ]:
# Статистика по ML признакам
print("📈 Статистика ML признаков:")
display(ml_features.describe())


## Batch обработка всех тикеров


In [ ]:
# Batch обработка ВСЕХ доступных тикеров
print("🚀 Запуск полного Feature Engineering Pipeline")
print(f"   Тикеры для обработки: {available_tickers}\n")

processed, errors = process_all_tickers(
    data_dir=DATA_DIR,
    output_ml_dir=OUTPUT_ML_DIR,
    output_backtest_dir=OUTPUT_BACKTEST_DIR,
    tickers=available_tickers
)

print(f"\n🏁 Pipeline завершен!")
print(f"   Успешно: {processed}/{len(available_tickers)}")
if errors:
    print(f"   Ошибки: {errors}")


## Проверка результатов


In [ ]:
# Проверка созданных файлов
print("📁 Файлы в processed_ml/:")
ml_files = list(OUTPUT_ML_DIR.glob("*.parquet"))
for f in ml_files:
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name}: {size_kb:.1f} KB")

print(f"\n📁 Файлы в backtest/:")
backtest_files = list(OUTPUT_BACKTEST_DIR.glob("*.parquet"))
for f in backtest_files:
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name}: {size_kb:.1f} KB")


In [ ]:
# Загрузка и проверка одного из созданных файлов
if ml_files:
    sample_file = ml_files[0]
    sample_df = pd.read_parquet(sample_file)
    print(f"📊 Пример: {sample_file.name}")
    print(f"   Строк: {len(sample_df)}")
    print(f"   Столбцов: {len(sample_df.columns)}")
    print(f"   Столбцы: {sample_df.columns.tolist()}")
    
    # Финальная проверка на запрещенные столбцы
    forbidden = [col for col in sample_df.columns if col in FORBIDDEN_ML_COLUMNS]
    if forbidden:
        print(f"\n❌ КРИТИЧЕСКАЯ ОШИБКА: В ML файле есть запрещенные столбцы: {forbidden}")
    else:
        print(f"\n✅ ML файл прошел валидацию - нет абсолютных значений цен/объема")


## Сводка признаков

### Признаки для ML модели (всего ~30 признаков):

| Категория | Признаки | Описание |
|-----------|----------|----------|
| **Доходность** | `log_return` | Логарифмическая дневная доходность |
| **Объем** | `volume_zscore_20/60`, `volume_ratio_20`, `volume_spike` | Z-score и относительный объем |
| **Volume Profile** | `vp_position`, `vp_width_pct`, `vp_above_va` | Нормализованные метрики VP |
| **Тренд (MA)** | `dist_to_sma_20/50/200`, `dist_to_ema_20/50` | Расстояние до MA в % |
| **Наклон MA** | `sma_20_slope_norm`, `sma_50_slope_norm` | Нормализованный наклон |
| **Momentum** | `momentum_10`, `momentum_20` | Log return за период |
| **RSI** | `rsi_14` | Уже нормализован (0-100) |
| **Позиция цены** | `price_position_ma`, `trend_signal`, `trend_strength` | Категориальные/нормализованные |
| **Календарь** | `day_of_week`, `day_of_month`, `week_of_month` | Временные признаки |
| **Конец месяца** | `is_month_end`, `is_month_start` | Бинарные флаги |
| **Gap** | `overnight_gap`, `overnight_gap_zscore` | Гэп открытия |
| **Метаданные** | `ticker_id`, `sector_id`, `sector_encoded`, `liquidity_rank`, `is_blue_chip` | Для глобальной модели |
